In [2]:
!pip install --upgrade boto3
!pip install --upgrade awswrangler
!pip install --upgrade pandas
!pip install --upgrade pyarrow
!pip install --upgrade sagemaker
!pip install --upgrade numpy


  Using cached boto3-1.37.13-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.37.13-py3-none-any.whl.metadata (5.7 kB)
Using cached boto3-1.37.13-py3-none-any.whl (139 kB)
Using cached botocore-1.37.13-py3-none-any.whl (13.4 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.36.3
    Uninstalling botocore-1.36.3:
      Successfully uninstalled botocore-1.36.3
  Attempting uninstall: boto3
    Found existing installation: boto3 1.36.3
    Uninstalling boto3-1.36.3:
      Successfully uninstalled boto3-1.36.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.37.13 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 

In [5]:
!pip install --upgrade --force-reinstall pyarrow awswrangler pyathena


  Using cached pyarrow-19.0.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached awswrangler-3.11.0-py3-none-any.whl.metadata (17 kB)
  Using cached boto3-1.37.13-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.37.13-py3-none-any.whl.metadata (5.7 kB)
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pyarrow-18.1.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached awswrangler-3.11.0-py3-none-any.whl (379 kB)
Using cached pyarrow-18.1.0-cp311-cp311-manylinux_2_28_x86_64.whl (40.1 MB)
Using cached boto3-1.37.13-py3-none-any.whl (139 kB)
Using cached botocore-1.37.13-py3-none-any.whl (13.4 MB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 131.2 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found exis

In [6]:
import boto3
import sagemaker
import awswrangler as wr

# sess = sagemaker.Session()
# bucket = sess.default_bucket()
# role = sagemaker.get_execution_role()
# region = boto3.Session().region_name

# Define AWS and S3 parameters
database_name = "cardiovale_db"
bucket = "cardiovale-solutions-datascience-pipeline"
region = boto3.Session().region_name

AttributeError: module 'pyarrow.lib' has no attribute 'Decimal32Type'

In [7]:
ingest_create_athena_db_passed = False

# Import PyAthena

[PyAthena](https://pypi.org/project/PyAthena/) is a Python DB API 2.0 (PEP 249) compliant client for Amazon Athena.

In [4]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-sagemaker-sql-execution 0.1.6 requires pyathena<4,>=3.3.0, but you have pyathena 2.1.0 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.


# Create Athena Database

In [10]:
database_name = "dsoaws"

Note: The databases and tables that we create in Athena use a data catalog service to store the metadata of your data. For example, schema information consisting of the column names and data type of each column in a table, together with the table name, is saved as metadata information in a data catalog. 

Athena natively supports the AWS Glue Data Catalog service. When we run `CREATE DATABASE` and `CREATE TABLE` queries in Athena with the AWS Glue Data Catalog as our source, we automatically see the database and table metadata entries being created in the AWS Glue Data Catalog.

In [11]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [12]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [13]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS dsoaws


In [14]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_623/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


# Verify The Database Has Been Created Succesfully

In [15]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_623/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [16]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [17]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


# Store Variables for the Next Notebooks

In [18]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path_tsv                        -> 's3://sagemaker-us-east-1-786782285170/amazon-revi
s3_public_path_tsv                         -> 's3://usd-mads-508/amazon-reviews-pds/tsv'
setup_dependencies_passed                  -> True
setup_iam_roles_passed                     -> True
setup_instance_check_passed                -> True
setup_s3_bucket_passed                     -> True


# Release Resources

In [19]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}